# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/GenAI/RAG/Knowledge Graphs with LightRAG

/content/drive/MyDrive/GenAI/RAG/Knowledge Graphs with LightRAG


In [ ]:
!pip install PyPDF2 pyvis aioboto3 ollama tiktoken nano_vectordb python-dotenv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install lightrag-hku==1.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.6/103.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
# import libraries
import os
from IPython.display import Markdown, display
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete
import PyPDF2
from google.colab import userdata

In [ ]:
# Set the OpenAI API key in the environment
api_key = userdata.get('genai_course')
os.environ['OPENAI_API_KEY'] = api_key

# Create a directory

In [ ]:
# Set the name for the directory
WORKING_DIR = "./working_dir"

if not os.path.exists(WORKING_DIR):
    os.makedirs(WORKING_DIR)

rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func= gpt_4o_complete
)

In [ ]:
# If you are using Colab / Jupyter, do this:
import nest_asyncio
nest_asyncio.apply()

# Data Preparation

* PDF => TXT
* Create RAG with TXT

In [ ]:
# Open the PDF and extract the text
pdf_text = []
with open("Greek-gods-story.pdf", "rb") as file:
  reader = PyPDF2.PdfReader(file)
  for page in reader.pages:
    pdf_text.append(page.extract_text())

In [ ]:
# Save (Write) a file with the data
with open("greek_gods.txt", "w") as file:
  for line in pdf_text:
    file.write(f"{line}\n")

In [ ]:
# Build the RAG
with open("greek_gods.txt", "r") as file:
  rag.insert(file.read())

Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.20s/batch]

Extracting entities from chunks:   0%|          | 0/3 [00:00<?, ?chunk/s]


Extracting entities from chunks:  33%|███▎      | 1/3 [00:12<00:25, 12.66s/chunk]


Extracting entities from chunks:  67%|██████▋   | 2/3 [00:19<00:09,  9.37s/chunk]


Extracting entities from chunks: 100%|██████████| 3/3 [00:27<00:00,  9.31s/chunk]

Inserting entities: 100%|██████████| 38/38 [00:00<00:00, 4511.92entity/s]

Inserting relationships: 100%|██████████| 19/19 [00:00<00:00, 7148.53relationship/s]

Generating embeddings: 100%|██████████| 2/2 [00:02<00:00,  1.12s/batch]

Processing batch 1: 100%|██████████| 1/1 [00:34<00:00, 34.68s/it]


# RAG

In [ ]:
# PErform Naive RAG
def perform_rag(query, mode): # Changed function name to avoid shadowing
  return Markdown(rag.query(query, # Now rag refers to the LightRAG object
                            param = QueryParam(mode = mode)))
query = "Who is Hermes"
perform_rag(query, "naive") # Call the new function name

Hermes is a god in Greek mythology, recognized as a son of Zeus. Known for his cunning and mischievous personality, Hermes was born in a cave and rapidly demonstrated his cleverness. As a newborn, he crafted the first musical instrument, the lyre, using a tortoise shell. Despite his youth, he displayed a penchant for trickery when he stole a herd of cattle from the god Apollo. However, instead of inciting conflict, Hermes managed to appease Apollo with his enchanting music played on the lyre, ultimately exchanging the instrument for the cattle. This relationship with Apollo highlights both his diplomatic charm and cleverness.

Hermes is often depicted wearing a winged hat and sandals, emphasizing his role as a swift messenger among the gods and mortals. This ability allowed him to facilitate communication between the divine and human worlds, making him an essential figure in numerous myths and stories within Greek mythology. Throughout his life, Hermes continued to embody his playful and resourceful nature, characteristics that remain central to his mythological persona.

In [ ]:
# Perform LightRAG with a local search
# This means we look into closer entities
perform_rag(query, "local")

```json
{
  "high_level_keywords": ["Greek mythology", "Hermes"],
  "low_level_keywords": ["Messenger of the gods", "Olympian", "Zeus", "Caduceus", "Travelers"]
}
```


### Who is Hermes?

Hermes is a figure from Greek mythology known for his cleverness and inventiveness. He is recognized as the son of Zeus and is famed for creating the first lyre, a musical instrument crafted from a tortoise shell and strings. This act of creation highlights Hermes' innovative spirit and his affinity for music.

### Role and Attributes

Hermes is commonly depicted with a winged hat and sandals, symbolizing his role as the messenger of the gods. These attributes allowed him to travel swiftly, facilitating communication between gods and mortals. Hermes' clever and mischievous nature often plays a significant part in mythological tales, adding to his dynamic persona.

### Relationship with Other Gods

- **Zeus**: Hermes shares a close relationship with his father, Zeus. They have been depicted traveling together in disguise, illustrating a trusted father-son dynamic.
  
- **Apollo**: Hermes' interaction with Apollo is a notable story where Hermes enchants him with the lyre's music. Eventually, Hermes trades the lyre with Apollo, securing an amicable exchange despite initially stealing Apollo's cattle.

### Influences and Legacy

Hermes is often seen as a divine trickster, utilizing his quick wit and resourcefulness in various myths. His innovation in creating the lyre and his activities as a messenger underscore his role in promoting communication and arts. Hermes' stories reflect his enduring legacy as a symbol of ingenuity and communication.

In [ ]:
# Perform LightRAG with Global Search
# More diverse results, global entity relationships
perform_rag(query, "global")

```json
{
  "high_level_keywords": ["Hermes", "Greek mythology", "Deity"],
  "low_level_keywords": ["Messenger of the gods", "Olympian god", "God of trade", "God of travel", "God of thievery", "Winged sandals"]
}
```


## Hermes: The Greek Messenger God

Hermes is a prominent figure in Greek mythology, known for his cleverness and inventiveness. He is the son of Zeus, the ruler of the Greek gods, establishing a notable bond with his father, who trusts him deeply. This father-son relationship is highlighted through their shared journeys, especially when both traveled in disguise across regions.

## Attributes and Inventions

One of the most distinctive features Hermes possesses is his winged hat and sandals, enabling him to move swiftly across the realms. As the messenger of the gods, these accessories are crucial for his role in conveying messages between gods and mortals. His inventive nature is further exemplified by his creation of the first lyre, a stringed musical instrument. The lyre’s creation was not just a testament to his ingenuity but also played a significant part in his encounters with other gods, such as enchanting Apollo with its music.

## Role in Mythology and Influence

Hermes is depicted as a mischievous figure throughout mythology, a trait that extends to his actions such as stealing Apollo's herd of cattle. Despite such antics, his intelligence and charm often turn these potentially negative actions to his advantage. This ability to navigate through situations cleverly means Hermes is not only a trickster but also a respected figure among gods.

In his capacity as a messenger, Hermes is essential to the flow of communication and interaction among the divine beings and their mortal counterparts. His role, being central to numerous Greek myths, underscores his importance in these ancient stories. Hermes' blend of playfulness, creativity, and his crucial role as a divine emissary capture the fascination that makes him a celebrated character in Greek mythology.

In [ ]:
# Perform LightRAG with Hybrid approach
perform_rag(query, "hybrid")

```json
{
  "high_level_keywords": ["Hermes"],
  "low_level_keywords": ["Greek mythology", "Messenger god", "Olympian gods", "Divine attributes", "Ancient Greece"]
}
```


## Hermes: The Agile Messenger of the Gods

### Background and Attributes

Hermes, a prominent figure in Greek mythology, is known for his cleverness, inventiveness, and agility. As the son of Zeus, Hermes inherited a blend of divine traits that set him apart in the pantheon of Greek gods. One of his notable inventions was the creation of the first lyre, crafted ingeniously from a tortoise shell and strings. This musical instrument not only showcases his creativity but also his ability to enchant others, as demonstrated in his interaction with Apollo.

Hermes is often depicted with his distinctive winged hat and sandals, which symbolize his role as the swift messenger of the gods. These elements of his attire emphasize his ability to travel quickly, making him an essential link between the divine and mortal worlds. His agility and resourcefulness are hallmarks of his character, allowing him to navigate and mediate various situations among gods and humans alike.

### Relationship with Zeus

Hermes shares a close and trusting relationship with his father, Zeus. This is exemplified by their shared journeys, often undertaken in disguise to test the hospitality and kindness of mortals. In one notable story, Zeus and Hermes reward the kindness of Baucis and Philemon by transforming their humble abode into a temple, highlighting the divine appreciation for human generosity and kindness.

### Influence on Myths

Hermes plays a significant role in several Greek myths, often acting as an intermediary or catalyst for events. His playful and mischievous nature, combined with his ingenuity, make him a central figure in many stories. His interactions often lead to both humorous and profound outcomes, impacting the lives of gods and mortals alike.

In summary, Hermes is a multifaceted deity revered for his intelligence, communication skills, and inventiveness. His contributions to Greek mythology showcase the interplay between divine intervention and human actions, making him an enduring character in these ancient narratives.

# Graph Visualization

In [ ]:
# Import libraries
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import random
from IPython.display import HTML

In [ ]:
# Load entities
G = nx.read_graphml(f"{WORKING_DIR}/graph_chunk_entity_relation.graphml")

# Create a network
net = Network(notebook = True, cdn_resources = "remote")
net.from_nx(G)

# Add colors and titles to nodes
for node in net.nodes:
  node["color"] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
  if "description" in node:
    node["title"] = node["description"]

# Add titles to edges
for edge in net.edges:
  if "description" in  edge:
    edge["title"] = edge["description"]

# Save and display the netweok
net.show("graph.html")
net.save_graph("graph.html")

HTML(filename = "graph.html")

graph.html


In [ ]:
# Print all node names
for node in G.nodes:
  print(node)

"KING OF SPARTA"
"HERA"
"ATHENS"
"BAUCIS"
"HUNGER"
"OLYMPUS"
"ALPHEUS"
"TROJAN WAR"
"THE GOLDEN APPLE"
"APOLLO"
"PHILEMON"
"SLAVES"
"HESTIA"
"HADES"
"THE TROJAN WAR"
"THE LYRE"
"THE TEMPLE"
"ERYSICHTHON"
"THESSALY"
"ORPHEUS"
"DIOMEDES"
"ARES"
"ATHENA"
"DEMETER"
"GOLDEN APPLE"
"HEPHAESTUS"
"PARIS"
"APHRODITE"
"POSEIDON"
"GODDESS FORM"
"THE UNDERWORLD"
"DIONYSUS"
"HELEN"
"ERIS"
"HERMES"
"ZEUS"
"MOUNT OLYMPUS"
"ARTEMIS"


In [ ]:
# Target Zeus
# Load entities
G = nx.read_graphml(f"{WORKING_DIR}/graph_chunk_entity_relation.graphml")

# Target Zeus
target_node = '"ZEUS"'

# Get neighbors for that node
subgraph_nodes = list(G.neighbors(target_node)) + [target_node]
subgraph = G.subgraph(subgraph_nodes)

# Create a network
net = Network(notebook = True, cdn_resources = "remote")
net.from_nx(subgraph)

# Save and display the netweok
graph_name = f"{target_node}_graph.html"
net.save_graph(graph_name)
HTML(filename = graph_name)